<a href="https://colab.research.google.com/github/Yashwanth-1406/Pyspark-yashwanth/blob/main/spark-04/08/2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
sql_path = '/content/drive/My Drive/mysqlsampledatabase.sql'

with open(sql_path, 'r', encoding='utf-8') as f:
    sql_content = f.read()

print(sql_content[:500])


/* Create the database */
CREATE DATABASE  IF NOT EXISTS classicmodels;

/* Switch to the classicmodels database */
USE classicmodels;

/* Drop existing tables  */
DROP TABLE IF EXISTS productlines;
DROP TABLE IF EXISTS products;
DROP TABLE IF EXISTS offices;
DROP TABLE IF EXISTS employees;
DROP TABLE IF EXISTS customers; 
DROP TABLE IF EXISTS payments;
DROP TABLE IF EXISTS orders;
DROP TABLE IF EXISTS orderdetails;

/* Create the tables */
CREATE TABLE productlines (
  productLine varchar(50),



In [ ]:
import re
import pandas as pd


product_inserts = re.findall(r"INSERT INTO products\s*\(.*?\)\s*VALUES\s*(.*?);", sql_content, re.DOTALL | re.IGNORECASE)


all_rows = []
for block in product_inserts:
    rows = re.findall(r'\((.*?)\)', block)
    for row in rows:
        fields = [x.strip().strip("'") for x in row.split(",")]
        all_rows.append(fields)

columns = ["productCode", "productName", "productLine", "productScale", "productVendor", "productDescription", "quantityInStock", "buyPrice", "MSRP"]


df = pd.DataFrame(all_rows, columns=columns)


csv_path = "/content/drive/My Drive/products.csv"
df.to_csv(csv_path, index=False)

print("CSV saved to:", csv_path)


CSV saved to: /content/drive/My Drive/products.csv


In [ ]:
json_path = "/content/drive/My Drive/products.json"
df.to_json(json_path, orient='records', lines=True)
print("JSON saved to:", json_path)


JSON saved to: /content/drive/My Drive/products.json


In [ ]:
df["buyPrice"] = df["buyPrice"].astype(float) * 1.10  # Increase by 10%
df.to_csv(csv_path, index=False)  # Save back to Drive


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

sql_path = '/content/drive/My Drive/mysqlsampledatabase.sql'

with open(sql_path, 'r', encoding='utf-8') as f:
    sql_content = f.read()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
sql_path = '/content/drive/My Drive/mysqlsampledatabase.sql'

with open(sql_path, 'r', encoding='utf-8') as f:
    sql_content = f.read()

print(sql_content[:500])  # Optional: Preview first 500 characters


/* Create the database */
CREATE DATABASE  IF NOT EXISTS classicmodels;

/* Switch to the classicmodels database */
USE classicmodels;

/* Drop existing tables  */
DROP TABLE IF EXISTS productlines;
DROP TABLE IF EXISTS products;
DROP TABLE IF EXISTS offices;
DROP TABLE IF EXISTS employees;
DROP TABLE IF EXISTS customers; 
DROP TABLE IF EXISTS payments;
DROP TABLE IF EXISTS orders;
DROP TABLE IF EXISTS orderdetails;

/* Create the tables */
CREATE TABLE productlines (
  productLine varchar(50),



In [ ]:
sql_path = '/content/drive/My Drive/mysqlsampledatabase.sql'

with open(sql_path, 'r', encoding='utf-8') as f:
    sql_content = f.read()


In [ ]:
import re
import pandas as pd

# Match the INSERT statement for 'products'
match = re.search(r"INSERT\s+INTO\s+products\s*\((.*?)\)\s*VALUES\s*(.*?);", sql_content, flags=re.DOTALL | re.IGNORECASE)

columns = [col.strip() for col in match.group(1).split(",")]
rows_raw = match.group(2)

# Split into rows inside VALUES
rows = re.findall(r"\((.*?)\)", rows_raw)

parsed_rows = []
for row in rows:
    # Split while handling quotes and commas inside strings
    parts = re.findall(r"'(.*?)'|(\bNULL\b|\b\d+\.\d+\b|\b\d+\b)", row)
    cleaned = []
    for val in parts:
        if val[0]:
            cleaned.append(val[0])  # It's a quoted string
        elif val[1] == 'NULL':
            cleaned.append(None)
        else:
            try:
                cleaned.append(float(val[1]) if '.' in val[1] else int(val[1]))
            except:
                cleaned.append(val[1])
    parsed_rows.append(cleaned)

# Convert to DataFrame
df = pd.DataFrame(parsed_rows, columns=columns)
df.head()


,productCode,productName,productLine,productScale,productVendor,productDescription,quantityInStock,buyPrice,MSRP
0,S10_1678,1969 Harley Davidson Ultimate Chopper,Motorcycles,1:10,Min Lin Diecast,"This replica features working kickstand, front...",7933,48.81,95.70


In [ ]:
import sqlite3

conn = sqlite3.connect(':memory:')
df.to_sql('products', conn, if_exists='replace', index=False)

# Check if data was loaded
pd.read_sql("SELECT * FROM products LIMIT 5", conn)


,productCode,productName,productLine,productScale,productVendor,productDescription,quantityInStock,buyPrice,MSRP
0,S10_1678,1969 Harley Davidson Ultimate Chopper,Motorcycles,1:10,Min Lin Diecast,"This replica features working kickstand, front...",7933,48.81,95.70


In [ ]:
df.to_csv('/content/drive/My Drive/products_clean.csv', index=False)
df.to_json('/content/drive/My Drive/products_clean.json', orient='records', lines=True)


In [ ]:
match = re.search(r"INSERT\s+INTO\s+customers\s*\((.*?)\)\s*VALUES\s*(.*?);", sql_content, flags=re.DOTALL | re.IGNORECASE)


In [ ]:
import re
import pandas as pd
import sqlite3
import csv
import io
from tqdm import tqdm
from google.colab import drive

# Step 1: Mount Drive (skip if already mounted)
drive.mount('/content/drive')

# Step 2: Load the SQL file
sql_path = '/content/drive/My Drive/mysqlsampledatabase.sql'
with open(sql_path, 'r', encoding='utf-8') as f:
    sql_content = f.read()

# Step 3: Extract all INSERT INTO blocks
insert_blocks = re.findall(
    r"INSERT\s+INTO\s+(\w+)\s*\((.*?)\)\s*VALUES\s*(.*?);",
    sql_content, re.DOTALL | re.IGNORECASE
)

# Step 4: Prepare SQLite in-memory DB
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

# Step 5: Process each INSERT block
for table_name, columns_str, values_block in tqdm(insert_blocks, desc="Processing tables"):

    columns = [col.strip() for col in columns_str.split(",")]

    # Extract each row
    row_matches = re.findall(r"\((.*?)\)", values_block)
    parsed_rows = []

    for row in row_matches:
        try:
            # Use csv.reader to safely handle commas inside quotes
            f = io.StringIO(row)
            reader = csv.reader(f, skipinitialspace=True, quotechar="'")
            parsed = next(reader)

            # Convert 'NULL' to None
            parsed = [None if val.upper() == 'NULL' else val for val in parsed]

            # ✅ Only add rows that match column count
            if len(parsed) == len(columns):
                parsed_rows.append(parsed)
            else:
                print(f"⚠️ Skipped row in `{table_name}`: column mismatch ({len(parsed)} vs {len(columns)})")

        except Exception as e:
            print(f"⚠️ Error parsing row in `{table_name}`: {e}")
            continue

    if not parsed_rows:
        print(f"❌ Skipping `{table_name}`: no valid rows.")
        continue

    # Create DataFrame
    try:
        df = pd.DataFrame(parsed_rows, columns=columns)
    except Exception as e:
        print(f"❌ DataFrame creation failed for `{table_name}`: {e}")
        continue

    # Save CSV and JSON to Drive
    csv_path = f"/content/drive/My Drive/{table_name}.csv"
    json_path = f"/content/drive/My Drive/{table_name}.json"
    df.to_csv(csv_path, index=False)
    df.to_json(json_path, orient='records', lines=True)

    # Load into SQLite
    try:
        df.to_sql(table_name, conn, if_exists='replace', index=False)
    except Exception as e:
        print(f"⚠️ SQLite insert failed for `{table_name}`: {e}")

    print(f"✅ {table_name}: {len(df)} rows → saved to Drive + SQLite")

print("🎉 ALL TABLES PROCESSED SUCCESSFULLY!")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Processing tables:  75%|███████▌  | 6/8 [00:00<00:00, 59.26it/s]

⚠️ Skipped row in `productlines`: column mismatch (6 vs 4)
✅ productlines: 6 rows → saved to Drive + SQLite
✅ products: 1 rows → saved to Drive + SQLite
✅ offices: 7 rows → saved to Drive + SQLite
✅ employees: 23 rows → saved to Drive + SQLite
⚠️ Skipped row in `customers`: column mismatch (5 vs 13)
⚠️ Skipped row in `customers`: column mismatch (5 vs 13)
⚠️ Skipped row in `customers`: column mismatch (5 vs 13)
⚠️ Skipped row in `customers`: column mismatch (6 vs 13)
⚠️ Skipped row in `customers`: column mismatch (5 vs 13)
⚠️ Skipped row in `customers`: column mismatch (5 vs 13)
⚠️ Skipped row in `customers`: column mismatch (5 vs 13)
⚠️ Skipped row in `customers`: column mismatch (5 vs 13)
⚠️ Skipped row in `customers`: column mismatch (5 vs 13)
⚠️ Skipped row in `customers`: column mismatch (5 vs 13)
⚠️ Skipped row in `customers`: column mismatch (5 vs 13)
⚠️ Skipped row in `customers`: column mismatch (5 vs 13)
⚠️ Skipped row in `customers`: column mismatch (14 vs 13)
⚠️ Skipped row

Processing tables: 100%|██████████| 8/8 [00:00<00:00, 41.26it/s]

✅ orderdetails: 2996 rows → saved to Drive + SQLite
🎉 ALL TABLES PROCESSED SUCCESSFULLY!


In [ ]:
# Example: query products
pd.read_sql("SELECT * FROM products LIMIT 5", conn)


,productCode,productName,productLine,productScale,productVendor,productDescription,quantityInStock,buyPrice,MSRP
0,S10_1678,1969 Harley Davidson Ultimate Chopper,Motorcycles,1:10,Min Lin Diecast,"This replica features working kickstand, front...",7933,48.81,95.70


In [ ]:
# Preview top 5 orders
pd.read_sql("SELECT * FROM orders LIMIT 5", conn)

# Preview customer names and cities
pd.read_sql("SELECT customerName, city FROM customers LIMIT 10", conn)


,customerName,city
0,Atelier graphique,Nantes
1,Signal Gift Stores,Las Vegas
2,"Australian Collectors, Co.",Melbourne
3,La Rochelle Gifts,Nantes
4,Baane Mini Imports,Stavern
5,Mini Gifts Distributors Ltd.,San Rafael
6,"Blauer See Auto, Co.",Frankfurt
7,Mini Wheels Co.,San Francisco
8,Land of Toys Inc.,NYC
9,"Volvo Model Replicas, Co",Luleå


In [ ]:
# Total customers per country
pd.read_sql("SELECT country, COUNT(*) as num_customers FROM customers GROUP BY country", conn)

# Top 5 selling products
pd.read_sql("""
SELECT productCode, SUM(quantityOrdered) AS total_sold
FROM orderdetails
GROUP BY productCode
ORDER BY total_sold DESC
LIMIT 5
""", conn)


,productCode,total_sold
0,S18_3232,1808
1,S18_1342,1111
2,S700_4002,1085
3,S18_3856,1076
4,S50_1341,1074


In [ ]:
conn = sqlite3.connect('/content/drive/My Drive/classicmodels.db')
print("✅ SQLite DB saved to your Drive!")


✅ SQLite DB saved to your Drive!


In [ ]:
import os
from pathlib import Path

# 📁 Create a dedicated output folder in your Drive
output_folder = "/content/drive/My Drive/classicmodels_data"
Path(output_folder).mkdir(parents=True, exist_ok=True)

# 💾 Save SQLite file too (optional)
sqlite_db_path = f"{output_folder}/classicmodels.db"
conn_file = sqlite3.connect(sqlite_db_path)

# 🔁 Process each table
for table_name, columns_str, values_block in tqdm(insert_blocks, desc="Processing tables"):

    columns = [col.strip() for col in columns_str.split(",")]
    row_matches = re.findall(r"\((.*?)\)", values_block)
    parsed_rows = []

    for row in row_matches:
        try:
            f = io.StringIO(row)
            reader = csv.reader(f, skipinitialspace=True, quotechar="'")
            parsed = next(reader)
            parsed = [None if val.upper() == 'NULL' else val for val in parsed]

            if len(parsed) == len(columns):
                parsed_rows.append(parsed)
            else:
                print(f"⚠️ Skipped row in `{table_name}`: column mismatch ({len(parsed)} vs {len(columns)})")

        except Exception as e:
            print(f"⚠️ Error parsing row in `{table_name}`: {e}")
            continue

    if not parsed_rows:
        print(f"❌ Skipping `{table_name}`: no valid rows.")
        continue

    try:
        df = pd.DataFrame(parsed_rows, columns=columns)
    except Exception as e:
        print(f"❌ DataFrame creation failed for `{table_name}`: {e}")
        continue

    # ✅ Save to CSV and JSON in new Drive folder
    csv_path = os.path.join(output_folder, f"{table_name}.csv")
    json_path = os.path.join(output_folder, f"{table_name}.json")
    df.to_csv(csv_path, index=False)
    df.to_json(json_path, orient='records', lines=True)

    # ✅ Also save to SQLite file
    try:
        df.to_sql(table_name, conn_file, if_exists='replace', index=False)
    except Exception as e:
        print(f"⚠️ SQLite insert failed for `{table_name}`: {e}")

    print(f"✅ {table_name}: {len(df)} rows → saved to folder & SQLite")

print("🎉 All tables processed and saved to Drive!")
print(f"📂 Data saved in folder: {output_folder}")
print(f"🧱 SQLite DB saved at: {sqlite_db_path}")


Processing tables:  25%|██▌       | 2/8 [00:00<00:00, 19.79it/s]

⚠️ Skipped row in `productlines`: column mismatch (6 vs 4)
✅ productlines: 6 rows → saved to folder & SQLite
✅ products: 1 rows → saved to folder & SQLite
✅ offices: 7 rows → saved to folder & SQLite
✅ employees: 23 rows → saved to folder & SQLite
⚠️ Skipped row in `customers`: column mismatch (5 vs 13)
⚠️ Skipped row in `customers`: column mismatch (5 vs 13)
⚠️ Skipped row in `customers`: column mismatch (5 vs 13)
⚠️ Skipped row in `customers`: column mismatch (6 vs 13)
⚠️ Skipped row in `customers`: column mismatch (5 vs 13)
⚠️ Skipped row in `customers`: column mismatch (5 vs 13)
⚠️ Skipped row in `customers`: column mismatch (5 vs 13)
⚠️ Skipped row in `customers`: column mismatch (5 vs 13)
⚠️ Skipped row in `customers`: column mismatch (5 vs 13)
⚠️ Skipped row in `customers`: column mismatch (5 vs 13)
⚠️ Skipped row in `customers`: column mismatch (5 vs 13)
⚠️ Skipped row in `customers`: column mismatch (5 vs 13)
⚠️ Skipped row in `customers`: column mismatch (14 vs 13)
⚠️ Skipped

Processing tables: 100%|██████████| 8/8 [00:00<00:00, 19.30it/s]

✅ customers: 100 rows → saved to folder & SQLite
⚠️ Skipped row in `orders`: column mismatch (6 vs 7)
✅ orders: 63 rows → saved to folder & SQLite
✅ payments: 273 rows → saved to folder & SQLite
✅ orderdetails: 2996 rows → saved to folder & SQLite
🎉 All tables processed and saved to Drive!
📂 Data saved in folder: /content/drive/My Drive/classicmodels_data
🧱 SQLite DB saved at: /content/drive/My Drive/classicmodels_data/classicmodels.db


In [ ]:
import os

output_folder = "/content/drive/My Drive/classicmodels_data"

# List files in the folder
for file in os.listdir(output_folder):
    print(file)


classicmodels.db
productlines.csv
productlines.json
products.csv
products.json
offices.csv
offices.json
employees.csv
employees.json
customers.csv
customers.json
orders.csv
orders.json
payments.csv
payments.json
orderdetails.csv
orderdetails.json


In [ ]:
files = [os.path.join(output_folder, f) for f in os.listdir(output_folder)]
for f in files:
    print(f)


/content/drive/My Drive/classicmodels_data/classicmodels.db
/content/drive/My Drive/classicmodels_data/productlines.csv
/content/drive/My Drive/classicmodels_data/productlines.json
/content/drive/My Drive/classicmodels_data/products.csv
/content/drive/My Drive/classicmodels_data/products.json
/content/drive/My Drive/classicmodels_data/offices.csv
/content/drive/My Drive/classicmodels_data/offices.json
/content/drive/My Drive/classicmodels_data/employees.csv
/content/drive/My Drive/classicmodels_data/employees.json
/content/drive/My Drive/classicmodels_data/customers.csv
/content/drive/My Drive/classicmodels_data/customers.json
/content/drive/My Drive/classicmodels_data/orders.csv
/content/drive/My Drive/classicmodels_data/orders.json
/content/drive/My Drive/classicmodels_data/payments.csv
/content/drive/My Drive/classicmodels_data/payments.json
/content/drive/My Drive/classicmodels_data/orderdetails.csv
/content/drive/My Drive/classicmodels_data/orderdetails.json


In [ ]:
import pandas as pd

df = pd.read_csv("/content/drive/My Drive/classicmodels_data/products.csv")
df.head()


,productCode,productName,productLine,productScale,productVendor,productDescription,quantityInStock,buyPrice,MSRP
0,S10_1678,1969 Harley Davidson Ultimate Chopper,Motorcycles,1:10,Min Lin Diecast,"This replica features working kickstand, front...",7933,48.81,95.7


In [ ]:
sqlite_path = "/content/drive/My Drive/classicmodels_data/classicmodels.db"
os.path.exists(sqlite_path)


True

In [ ]:
import sqlite3

conn = sqlite3.connect(sqlite_path)
pd.read_sql("SELECT name FROM sqlite_master WHERE type='table'", conn)


,name
0,productlines
1,products
2,offices
3,employees
4,customers
5,orders
6,payments
7,orderdetails


In [ ]:
from IPython.display import FileLink

for file in os.listdir(output_folder):
    path = os.path.join(output_folder, file)
    display(FileLink(path))


/content/drive/My Drive/classicmodels_data/classicmodels.db

/content/drive/My Drive/classicmodels_data/productlines.csv

/content/drive/My Drive/classicmodels_data/productlines.json

/content/drive/My Drive/classicmodels_data/products.csv

/content/drive/My Drive/classicmodels_data/products.json

...

/content/drive/My Drive/classicmodels_data/orders.json

/content/drive/My Drive/classicmodels_data/payments.csv

/content/drive/My Drive/classicmodels_data/payments.json

/content/drive/My Drive/classicmodels_data/orderdetails.csv

/content/drive/My Drive/classicmodels_data/orderdetails.json